# Find and replace the given text in the PDF file

- Used PyMuPDF because it supports element level coordinates

In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 22.7 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF

def find_text_in_pdf(pdf_path, search_text):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    results = []

    # Iterate through the pages
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text_instances = page.search_for(search_text)

        # If the text is found, store the result with page number and rectangle coordinates
        for instance in text_instances:
            results.append({
                "page": page_num + 1,
                "coordinates": instance
            })

    pdf_document.close()

    return results

# Example usage
pdf_path = "/content/pdf_files/in/research_paper.pdf"
search_text = "research paper"
locations = find_text_in_pdf(pdf_path, search_text)

if locations:
    print(f"Text '{search_text}' found at:")
    for location in locations:
        print(f"Page {location['page']}, Coordinates: {location['coordinates']}")
else:
    print(f"Text '{search_text}' not found in the PDF.")


Text 'research paper' found at:
Page 1, Coordinates: Rect(85.10399627685547, 70.81623077392578, 286.26788330078125, 98.8174819946289)


In [ ]:
import fitz  # PyMuPDF

def int_to_rgb(color_int):
    # Convert an integer color to RGB format (tuple of floats in range 0-1)
    r = (color_int >> 16) & 255
    g = (color_int >> 8) & 255
    b = color_int & 255
    return (r / 255.0, g / 255.0, b / 255.0)

def replace_text_with_x_in_pdf(input_pdf_path, output_pdf_path, search_text):
    # Open the PDF file
    pdf_document = fitz.open(input_pdf_path)

    is_modified = False

    # Iterate through the pages
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text_instances = page.search_for(search_text)

        if not text_instances:
            print(f"Text '{search_text}' not found on page {page_num + 1}. Skipping...")
            continue

        # If the text is found, replace each character with 'X', keeping original styling
        for instance in text_instances:
            # Extract text within the bounding box
            words = page.get_text("words", clip=instance)
            for word in words:
                word_rect = fitz.Rect(word[:4])
                original_text = word[4]
                replacement_text = 'X' * len(original_text)

                # Get original font size and color by using the first character
                spans = page.get_text("dict", clip=word_rect)["blocks"][0]["lines"][0]["spans"]
                font_size = spans[0]["size"]
                color_int = spans[0]["color"]

                # Convert the integer color value to RGB format
                rgb_color = int_to_rgb(color_int)

                # Draw a white rectangle to cover the original text
                page.draw_rect(word_rect, color=(1, 1, 1), fill=(1, 1, 1), fill_opacity=1)

                # Ensure the replacement text is placed correctly
                page.insert_text(word_rect.tl, replacement_text, fontsize=font_size, color=rgb_color)

            is_modified = True

    if not is_modified:
        print(f"No occurrences of '{search_text}' found in the PDF.")
        return

    # Save the modified PDF to a new file
    print(f"Replaced '{search_text}' with 'X' and saved to '{output_pdf_path}'.")
    pdf_document.save(output_pdf_path)
    pdf_document.close()

# Example usage
input_pdf_path = "/content/pdf_files/in/research_paper.pdf"
output_pdf_path = "/content/pdf_files/out/research_paper_v4.pdf"
search_text = "research paper"

replace_text_with_x_in_pdf(input_pdf_path, output_pdf_path, search_text)


Text 'research paper' not found on page 2. Skipping...
Text 'research paper' not found on page 3. Skipping...
Text 'research paper' not found on page 4. Skipping...
Text 'research paper' not found on page 5. Skipping...
Text 'research paper' not found on page 6. Skipping...
Text 'research paper' not found on page 7. Skipping...
Text 'research paper' not found on page 8. Skipping...
Text 'research paper' not found on page 9. Skipping...
Text 'research paper' not found on page 10. Skipping...
Text 'research paper' not found on page 11. Skipping...
Replaced 'research paper' with 'X' and saved to '/content/pdf_files/out/research_paper_v4.pdf'.
